In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import validation_curve
import pickle
import sklearn.model_selection as ms

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('../data/datasets/cleanDataConfig.csv', sep=';')
df.head()

,id_leg,operation,runway,hexid,callsign,type,origin,destination,altitude,ground_speed,vertical_rate,tmp,dew_pt,rel_hum,wind_dir,win_sp,wind_gust,visib,press,sky_lvl,rain,snow,drizzle,fog,time_class,day_week,month,time_ref,operation_Landing,operation_TakeOff,runway_14L,runway_14R,runway_18L,runway_18R,runway_32L,runway_32R,runway_36L,runway_36R,configuration_NORTE,configuration_SUR,configuration
0,733513,0,5,483,6920,22,186,268,1625.0,142.0,-768.0,7.0,0.0,61.017,250.0,20.372,0.000000,10.0,1030.0,10.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0,0
1,733498,0,5,515,246,23,664,268,1850.0,137.0,-768.0,7.0,-1.0,56.724,250.0,25.928,44.447962,10.0,1030.0,10.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0,0
2,733495,0,5,814,262,67,212,268,2200.0,156.0,-832.0,7.0,-1.0,56.724,230.0,16.668,0.000000,10.0,1031.0,10.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0,0
3,733501,0,5,625,277,23,254,268,1862.5,143.0,-640.0,7.0,-1.0,56.724,230.0,16.668,0.000000,10.0,1031.0,10.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0,0
4,733496,0,5,491,268,23,421,268,2000.0,136.0,-640.0,5.0,0.0,70.063,160.0,12.964,0.000000,10.0,1031.0,10.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0,0


In [3]:
df.shape

(733822, 41)

In [4]:
columns_names = df.columns.to_list()

predictors = columns_names[:38]

target = columns_names[-1]

X = df[predictors]

Y = df[target]

In [5]:
X_train , X_test , Y_train , Y_test = train_test_split(X , Y , test_size=0.2, shuffle=True, random_state=0)

In [6]:
def randomForest(jobs=7):
    depht = [1,2,3,4,5,7,10,13,15,18,None]
    min_samples_split = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    min_samples_leaf = [0.1, 0.2, 0.3, 0.4, 0.5]

    print("RANDOM FOREST: INICIO PROFUNDIDAD")
    forest = RandomForestClassifier(n_jobs=jobs, n_estimators=200)
    
    # Obtenemos la curva de validacion variando la profundidad
    train_scores, test_scores = validation_curve(
        estimator = forest,
        X = X_train,
        y = Y_train,
        param_name='max_depth',
        param_range=depht,
        cv=ms.StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    )

    data = [train_scores, test_scores]

    with open('../data/serialized/randomForestPROFUNDIDAD.sav', 'wb') as file:
        pickle.dump(data, file, pickle.HIGHEST_PROTOCOL)

    with open('../data/resultados/curvaValidacion/resultados.csv', 'a', encoding='utf-8') as file:
        file.write(f"RANDOM_FOREST;PROFUNDIDAD;{train_scores.mean()};{test_scores.mean()}\n")


    print("RANDOM FOREST: PROFUNDIDAD TERMINADA")
    print("RANDOM FOREST: INICIO MIN SAMPLES SPLIT")

    # Obtenemos la curva de validacion variando el numero minimo de ejemplos para dividir un nodo
    train_scores, test_scores = validation_curve(
        estimator = forest,
        X = X_train,
        y = Y_train,
        param_name='min_samples_split',
        param_range=min_samples_split,
        cv=ms.StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    )

    data = [train_scores, test_scores]

    with open('../data/serialized/randomForestEJEMPLOS_SPLIT.sav', 'wb') as file:
        pickle.dump(data, file, pickle.HIGHEST_PROTOCOL)

    with open('../data/resultados/curvaValidacion/resultados.csv', 'a', encoding='utf-8') as file:
        file.write(f"RANDOM_FOREST;MIN_SAMPLES_SPLIT;{train_scores.mean()};{test_scores.mean()}\n")


    print("RANDOM FOREST: MIN SAMPLES SPLIT TERMINADA")
    print("RANDOM FOREST: INICIO MIN min_samples_leaf")

    # Obtenemos la curva de validacion variando el numero minimo de ejemplos por hoja
    train_scores, test_scores = validation_curve(
        estimator = forest,
        X = X_train,
        y = Y_train,
        param_name='min_samples_leaf',
        param_range=min_samples_leaf,
        cv=ms.StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    )

    data = [train_scores, test_scores]

    with open('../data/serialized/randomForestEJEMPLOS_HOJAS.sav', 'wb') as file:
        pickle.dump(data, file, pickle.HIGHEST_PROTOCOL)

    with open('../data/resultados/curvaValidacion/resultados.csv', 'a', encoding='utf-8') as file:
        file.write(f"RANDOM_FOREST;MIN_SAMPLES_LEAF;{train_scores.mean()};{test_scores.mean()}\n")


    print("RANDOM FOREST: min_samples_leaf TERMINADO")


In [7]:
def knn(jobs=7):
    distances = ['euclidean','manhattan', 'chebyshev', 'minkowski']
    vecinos = [5,7,9]

    for idx, vecino in enumerate(vecinos):    
        print(f"KNN ==> Vecino {idx+1} de {len(vecinos)}")
        # Obtenemos la curva de validacion variando los vecinos y el calculo de la distancia
        knn = KNeighborsClassifier(n_neighbors=vecino, n_jobs=jobs)
        train_scores, test_scores = validation_curve(
            estimator = knn
            X = X_train,
            y = Y_train,
            param_name='metric',
            param_range=distances,
            cv=ms.StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
        )

        data = [train_scores, test_scores]

        with open(f'../data/serialized/knnVC{vecino}.sav', 'wb') as file:
            pickle.dump(data, file, pickle.HIGHEST_PROTOCOL)

        with open('../data/resultados/curvaValidacion/resultados.csv', 'a', encoding='utf-8') as file:
            file.write(f"KNN;METRIC;{train_scores.mean()};{test_scores.mean()}\n")

In [8]:
def rrnn(jobs=7):
    print("Inicio RRNN")
    epocas = [10,20,50,100,150,200]

    print("INICIO RED NEURONAL")

    rrnn = MLPClassifier(hidden_layer_sizes=(38,25,40,35), random_state=0)
    
    # Obtenemos la curva de validacion variando el numero de epocas
    train_scores, test_scores = validation_curve(
        estimator = rrnn,
        X = X_train,
        y = Y_train,
        param_name='max_iter',
        param_range=epocas,
        cv=ms.StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    )

    data = [train_scores, test_scores]

    with open('../data/serialized/rrNN-DEF.sav', 'wb') as file:
        pickle.dump(data, file, pickle.HIGHEST_PROTOCOL)

In [9]:
def main(jobs=7):
    try:
        print("COMIENZO RANDOM FOREST")
        randomForest(jobs)
    except Exception as e:
        print(f"ERROR en el RANDOM FOREST. ERROR \n {e}")
        pass
    try:
        print("COMIENZO KNN")
        knn(jobs)
    except Exception as e:
        print(f"ERROR en el KNN. ERROR \n {e}")
        pass    
    try:
        print("COMIENZO RNNN")
        rrnn(jobs)
    except Exception as e:
        print(f"ERROR en el KNN. ERROR \n {e}")
        pass 
    print("FINNNNN!!!!!!!!!!!!")    
    

In [10]:
main()

COMIENZO RANDOM FOREST
RANDOM FOREST: INICIO PROFUNDIDAD
RANDOM FOREST: PROFUNDIDAD TERMINADA
RANDOM FOREST: INICIO MIN SAMPLES SPLIT
RANDOM FOREST: MIN SAMPLES SPLIT TERMINADA
RANDOM FOREST: INICIO MIN min_samples_leaf
RANDOM FOREST: min_samples_leaf TERMINADO
COMIENZO KNN
KNN ==> Vecino 1 de 3
KNN ==> Vecino 2 de 3
KNN ==> Vecino 3 de 3
COMIENZO RNNN
Inicio RRNN
INICIO RED NEURONAL
FINNNNN!!!!!!!!!!!!
